In [1]:
# Import necessary libraries.
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [2]:
# URL of a wikipedia which contains data of Falcon 9 and Falcon Heavy launches.
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

##### Request the Falcon9 launch Wiki page from its URL.

In [3]:
# Use requests 
response = requests.get(static_url, headers = headers)

In [4]:
# Create BEAUTIFULSOUP object from response.
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
# Print the page title to verify object's proper creation.
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
